<a href="https://colab.research.google.com/github/forexms78/AI-05-/blob/main/%EB%AF%B8%EC%85%986_7%ED%8C%80_%EB%B0%95%EB%B3%91%ED%98%B8.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

사전설정 토치 info 여러개 쓸거같은 라이브러리 임포트 따로해놓기
device 설정
구글 드라이브 설정 (데이터를 임포트해도댐)
캐글에서 다운받은 데이터 확인
데이터를 구글드라이브에 올리기 데이터가있으면 삭제 없으면 데이터를 복사
validation 파일 보강
데이터 전처리 이미지 크기조절 노말라이즈 0.5 0.5
데이터셋
데이터 sampler를 위한 가중치 생성함수
sampler를 이용한 dataloader

In [40]:
from urllib.request import urlretrieve; urlretrieve("https://raw.githubusercontent.com/c0z0c/jupyter_hangul/refs/heads/beta/helper_c0z0c_dev.py", "helper_c0z0c_dev.py")
import helper_c0z0c_dev as helper

In [41]:
!pip install -q kaggle

In [42]:
from google.colab import files
files.upload()

Saving kaggle.json to kaggle (1).json


{'kaggle (1).json': b'{"username":"bellboy1","key":"936ffa8a7fc031acd1ab553ee5fc9cec"}'}

In [43]:
!mkdir -p ~/.kaggle
!cp kaggle.json ~/.kaggle/

In [44]:
!chmod 600 ~/.kaggle/kaggle.json

In [45]:
DATASET_ID = 'paultimothymooney/chest-xray-pneumonia'
!kaggle datasets download -d $DATASET_ID

Dataset URL: https://www.kaggle.com/datasets/paultimothymooney/chest-xray-pneumonia
License(s): other
100% 2.29G/2.29G [00:32<00:00, 272MB/s]
100% 2.29G/2.29G [00:32<00:00, 75.0MB/s]


In [46]:
ZIP_FILE_NAME = 'chest-xray-pneumonia.zip'
TARGET_DIR = 'chest_xray_data'

In [ ]:
print(f"파일 압축을 해제")
!mkdir -p $TARGET_DIR
!unzip -q $ZIP_FILE_NAME -d $TARGET_DIR
!rm $ZIP_FILE_NAME

print("데이터 폴더")
!ls $TARGET_DIR/chest_xray

파일 압축을 해제
replace chest_xray_data/chest_xray/__MACOSX/._chest_xray? [y]es, [n]o, [A]ll, [N]one, [r]ename: 

In [ ]:
import torch
import torch.nn as nn
import torch.optim as optim

from torchvision import datasets, transforms
from torch.utils.data import DataLoader

import os
import matplotlib.pyplot as plt
import numpy as np
import pandas as pd
import seaborn as sns

In [ ]:
BASE_DIR = 'chest_xray_data/chest_xray'
TRAIN_DIR = os.path.join(BASE_DIR, 'train')
TEST_DIR = os.path.join(BASE_DIR, 'test')
VAL_DIR = os.path.join(BASE_DIR, 'val')

IMG_SIZE = 224
BATCH_SIZE = 32

In [ ]:
DEVICE = torch.device("cuda" if torch.cuda.is_available() else "cpu")

In [ ]:
# 정규화 진행

NORM_MEAN = [0.485, 0.456, 0.406]
NORM_STD = [0.229, 0.224, 0.225]

In [ ]:
# train 데이터 전처리 및 증강

train_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE, IMG_SIZE)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(10),
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN, std=NORM_STD)
])

# test, val 데이터 전처리

test_val_transforms = transforms.Compose([
    transforms.Resize((IMG_SIZE,IMG_SIZE)),
    transforms.ToTensor(),
    transforms.Normalize(mean=NORM_MEAN, std=NORM_STD)
])

In [ ]:
# 파이토치 데이터셋 생성

train_dataset = datasets.ImageFolder(TRAIN_DIR, transform=train_transforms)
test_dataset =  datasets.ImageFolder(TEST_DIR, transform=test_val_transforms)
val_dataset = datasets.ImageFolder(VAL_DIR, transform=test_val_transforms)

In [ ]:
# val 데이터가 적음으로 train test데이터랑 섞기

full_test_dataset = torch.utils.data.ConcatDataset([test_dataset, val_dataset])

In [ ]:
# 데이터 셋 크기확인

print(f'train : {len(train_dataset)}')
print(f'test : {len(full_test_dataset)}')
print(f'class : {test_dataset.class_to_idx}')

In [ ]:
# 데이터 로더 생성

train_loader = DataLoader(
    train_dataset,
    batch_size=BATCH_SIZE,
    shuffle=True,
    num_workers=4
)


test_loader = DataLoader(
    full_test_dataset,
    batch_size=BATCH_SIZE,
    shuffle=False,
    num_workers=4
)

In [ ]:
# 데이터 불균형이 있는 데이터이기때문에 이미지 갯수를 파악

pneumonia_count = sum(train_dataset.targets)

total_count = len(train_dataset)
normal_count = total_count - pneumonia_count

print(f'정상 이미지 : {normal_count}개')
print(f'폐렴 이미지 : {pneumonia_count}개')

class_labels = train_dataset.classes
class_counts = [normal_count, pneumonia_count]
df_counts = pd.DataFrame({'Class': class_labels, 'Count': class_counts})